In [1]:
# Install necessary packages if not already installed
# !pip install ipykernel --quiet
# !pip install ipywidgets --quiet
!pip install pandas --quiet
!pip install scikit-learn --quiet
!pip install matplotlib --quiet
!pip install seaborn --quiet
!pip install xgboost --quiet

In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold 
from sklearn.metrics import log_loss, f1_score 
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import xgboost as xgb

import warnings
warnings.filterwarnings("ignore")

le = LabelEncoder()

In [5]:


KMERS_TRAIN_PATH = '../data_prep/Data/ProcessedFiles/train_features_with_kmers_new.csv'
KMERS_TEST_PATH = '../data_prep/Data/ProcessedFiles/test_features_with_kmers_new.csv'
ORIGINAL_TRAIN_PATH = '../data_prep/Data/Train.csv'
ORIGINAL_TEST_PATH = '../data_prep/Data/SampleSubmission.csv'

train = pd.read_csv(KMERS_TRAIN_PATH)
train['ID'] = train['file'].apply(lambda x: x.split('.')[0])
test = pd.read_csv(KMERS_TEST_PATH)
test['ID'] = test['file'].apply(lambda x: x.split('.')[0])
train_labels = pd.read_csv(ORIGINAL_TRAIN_PATH)
train_labels['ID'] = train_labels['filename'].apply(lambda x: x.split('.')[0])

train = train.merge(train_labels[['ID', 'SampleType']], on='ID', how='left')
display(train.head(), train.shape, test.head(),test.shape)

,file,num_reads,avg_read_len,gc_content,q20_fraction,q30_fraction,A,T,G,C,...,k5_AGCGC,k5_GGTTT,k5_TTGGT,k5_AACCT,k5_GTTCG,k5_TTATC,k5_GCTTG,k5_ACTCC,ID,SampleType
0,ID_AAFNOT.fastq,42840,124.5,0.552629,0.949814,0.895745,0.182194,0.265177,0.268545,0.284083,...,0.001373,0.001094,0.000115,0.000359,0.004372,0.001242,0.000102,0.000141,ID_AAFNOT,Skin
1,ID_AAXPTO.fastq,42814,400.0,0.539853,0.986674,0.938658,0.268928,0.191218,0.316762,0.223092,...,0.000836,0.000995,0.006264,0.005534,0.000079,0.001557,0.002503,0.006719,ID_AAXPTO,Stool
2,ID_AAYKAN.fastq,21794,400.0,0.543314,0.984792,0.931714,0.264808,0.191878,0.318555,0.224759,...,0.002359,0.000493,0.004963,0.005346,0.000079,0.000741,0.002595,0.008116,ID_AAYKAN,Stool
3,ID_ABEZNS.fastq,448014,124.5,0.529188,0.961127,0.865468,0.203929,0.266883,0.255616,0.273572,...,0.002423,0.002742,0.000008,0.000342,0.003143,0.002430,0.000686,0.000341,ID_ABEZNS,Skin
4,ID_ABFFLP.fastq,15173,400.0,0.558125,0.991521,0.956243,0.261845,0.180030,0.330417,0.227708,...,0.001079,0.003701,0.004899,0.005040,0.000491,0.002871,0.003332,0.006180,ID_ABFFLP,Stool


(2900, 604)

,file,num_reads,avg_read_len,gc_content,q20_fraction,q30_fraction,A,T,G,C,...,k5_AGGCA,k5_AGCGC,k5_GGTTT,k5_TTGGT,k5_AACCT,k5_GTTCG,k5_TTATC,k5_GCTTG,k5_ACTCC,ID
0,ID_AMSOXU.fastq,3744,124.5,0.546307,0.965012,0.923914,0.193455,0.260238,0.264337,0.281970,...,0.000008,0.002992,0.002255,0.001100,0.003505,0.003785,0.002034,0.002385,0.001068,ID_AMSOXU
1,ID_AGKIYB.fastq,5050,124.5,0.550994,0.952111,0.898200,0.183995,0.265011,0.267772,0.283222,...,0.000088,0.001640,0.002030,0.000035,0.001542,0.004242,0.000611,0.000030,0.000800,ID_AGKIYB
2,ID_AOQGWN.fastq,7664,124.5,0.521396,0.967360,0.919571,0.210537,0.268067,0.250872,0.270524,...,0.000002,0.004998,0.004971,0.000108,0.000271,0.001344,0.004756,0.000176,0.000132,ID_AOQGWN
3,ID_ASNCNV.fastq,11756,124.5,0.554178,0.951195,0.869565,0.180014,0.265808,0.269669,0.284509,...,0.000033,0.001135,0.001451,0.000114,0.000636,0.004537,0.001002,0.000057,0.000395,ID_ASNCNV
4,ID_AVMUKC.fastq,3730,124.5,0.562520,0.945804,0.827714,0.172284,0.265196,0.274858,0.287662,...,0.000012,0.000377,0.000317,0.000027,0.000066,0.005270,0.000344,0.000024,0.000066,ID_AVMUKC


(1068, 603)

### Feature Selection

In [6]:
id_cols = ['file', 'filename', 'ID']
target_col = 'SampleType'
stat_cols = ['num_reads', 'avg_read_len', 'gc_content', 'q20_fraction', 'q30_fraction']
k4_cols = [col for col in train.columns if 'k4' in col]
k3_cols = [col for col in train.columns if 'k3' in col]
k2_cols = [col for col in train.columns if 'k2' in col]
k5_cols = [col for col in train.columns if 'k5' in col]
features =stat_cols+ k5_cols  + ['A',
 'T',
 'G',
 'C',]
print(features)
print(len(features))

['num_reads', 'avg_read_len', 'gc_content', 'q20_fraction', 'q30_fraction', 'k5_AGCGT', 'k5_TTCGC', 'k5_GGAAT', 'k5_GTAGG', 'k5_GGGTG', 'k5_ATTGG', 'k5_ACTGG', 'k5_TTGGG', 'k5_CTTCG', 'k5_GAGCG', 'k5_GAATT', 'k5_CGTAG', 'k5_TTTCG', 'k5_ACGGG', 'k5_CTTTC', 'k5_GGCGT', 'k5_AAAGG', 'k5_GCGTT', 'k5_GGAAA', 'k5_TAAAG', 'k5_GCTTT', 'k5_GGCGG', 'k5_TACGG', 'k5_ATTCC', 'k5_GCGTC', 'k5_ACGCT', 'k5_GGGCG', 'k5_GAAAG', 'k5_ACCGC', 'k5_GCGTA', 'k5_CCGGA', 'k5_CTTAA', 'k5_TATTG', 'k5_GCTCC', 'k5_GCCTT', 'k5_TGGGC', 'k5_CACAT', 'k5_CGGAA', 'k5_CCTTC', 'k5_GCGTG', 'k5_CGCAT', 'k5_CTGAG', 'k5_TCCCC', 'k5_TGGGG', 'k5_CGAGC', 'k5_CCTGA', 'k5_GTCAG', 'k5_CGGCC', 'k5_CGCTC', 'k5_CTCCT', 'k5_GCTTA', 'k5_TCCGG', 'k5_TGAGT', 'k5_GGGCT', 'k5_AGGAA', 'k5_CTCCC', 'k5_AACTG', 'k5_GCGCA', 'k5_TCCTC', 'k5_GAACG', 'k5_TGGAA', 'k5_CCCAC', 'k5_CGCGT', 'k5_GGTGA', 'k5_GCATT', 'k5_CGTCA', 'k5_TTCCT', 'k5_GTAAA', 'k5_CGGGT', 'k5_CAGCG', 'k5_TGAGA', 'k5_GTGAG', 'k5_AGGGT', 'k5_AAGTC', 'k5_CGGAT', 'k5_GGCTT', 'k5_TGTTC', 

### Cross Validation

In [7]:
### Cross Validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
X = train[features]
y = train[target_col]


In [ ]:
# -------------------------
# Retrain with best params
# -------------------------
test_fold_preds = []
logloss_scores = []
best_params =  {'learning_rate': 0.02906118152230375, 'max_depth': 11, 'min_child_weight': 2, 'subsample': 0.5471809612034713, 'colsample_bytree': 0.8656215107130768, 'lambda': 0.17474568803133841, 'alpha': 0.009532338933614777}
# best_params = {'learning_rate': 0.10693231492067892, '
for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
    print(f"------------------- FOLD {fold} --------------------------")
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    y_train = le.fit_transform(y_train)
    y_val = le.transform(y_val)

    best_params = best_params.copy()
    best_params.update({
        "objective": "multi:softprob",
        "num_class": len(np.unique(y)),
        "eval_metric": "mlogloss",
        "verbosity": 0,
        "tree_method": "hist",  # fast training, change if GPU available
        "random_state": 42,
        "use_label_encoder": False,
    })


    model = xgb.XGBClassifier(**best_params, n_estimators=1000)

    model.fit(
        X_train,
        y_train,
        eval_set=[(X_val, y_val)],

        verbose=False,
    )

    val_preds = model.predict_proba(X_val)
    val_logloss = log_loss(y_val, val_preds)
    logloss_scores.append(val_logloss)
    print(f"Fold {fold} Log Loss: {val_logloss:.5f}")
    print("-"*25)

    test_fold_preds.append(model.predict_proba(test[features]))

print(f"Average Log Loss: {np.mean(logloss_scores):.5f}")

# -------------------------
# Final blended predictions
# -------------------------
final_test_preds = np.mean(test_fold_preds, axis=0)
class_names = ["Mouth", "Nasal", "Skin", "Stool"]

# Normal probabilities submission
test[class_names] = final_test_preds
output = test[["ID"] + class_names]
os.makedirs("Subs", exist_ok=True)
output.to_csv("Subs/xgb_submission_10_folds_0_02154.csv", index=False)



------------------- FOLD 1 --------------------------
Fold 1 Log Loss: 0.00621
-------------------------
------------------- FOLD 2 --------------------------
Fold 2 Log Loss: 0.02003
-------------------------
------------------- FOLD 3 --------------------------
Fold 3 Log Loss: 0.00789
-------------------------
------------------- FOLD 4 --------------------------
Fold 4 Log Loss: 0.02529
-------------------------
------------------- FOLD 5 --------------------------
Fold 5 Log Loss: 0.07419
-------------------------
------------------- FOLD 6 --------------------------
Fold 6 Log Loss: 0.00658
-------------------------
------------------- FOLD 7 --------------------------
Fold 7 Log Loss: 0.01348
-------------------------
------------------- FOLD 8 --------------------------
Fold 8 Log Loss: 0.01774
-------------------------
------------------- FOLD 9 --------------------------
Fold 9 Log Loss: 0.01596
-------------------------
------------------- FOLD 10 --------------------------
